In [ ]:
from time import time
from os import path
from gensim import corpora, models
from gensim.models.phrases import Phrases
from corputil import FileCorpus, MultiFileCorpus
from corputil.modifiers import to_words_sl, sentence_to_words
from pprint import pprint
from collections import defaultdict

In [ ]:
label = 'KW45'
num_topics = 10
# chunksize, iterations, passes = 500, 500, 10
chunksize, iterations, passes = 200, 500, 20
file = path.join('data', 'Corpus_{}.txt'.format(label))

train = FileCorpus(file, modifier=to_words_sl)

def train_lda(train):
    corpus = [trigram[bigram[doc]] for doc in train]

    dictionary = corpora.Dictionary(corpus)
    dictionary.filter_extremes()
    dictionary.compactify()

    mmCorpus = [dictionary.doc2bow(doc) for doc in corpus]
    lda = models.LdaMulticore(mmCorpus, id2word=dictionary, chunksize=chunksize, batch=True,
                              num_topics=num_topics, workers=2, passes=passes, iterations=iterations)
    return lda

In [ ]:
bigram = Phrases.load(path.join('models', 'Bigram.phrase'))
trigram = Phrases.load(path.join('models', 'Trigram.phrase'))
model = train_lda(train)

In [ ]:
for topic in model.print_topics(num_topics):
    pprint(topic)